# 프로그래밍 과제 1

**이 노트북을 완성하여 LMS로 제출한다. 작성한 코드의 실행결과를 지우지 말고 보존한 상태로 제출해야 한다.**

In [ ]:
%matplotlib inline

In [ ]:
import os
import torch
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

10종류의 음식에 대해서 각각 1,000장의 이미지로 구성된 [데이터셋](https://drive.google.com/file/d/1rZ9DX4cDW7g0OGeFgkFKFzA_jZ1bzS93/view?usp=sharing)을 다운로드 받는다. 이 데이터셋은 torchvision이 제공하는 Food101 데이터셋에서 10종류의 음식만 선택해서 만든 것이다. 다운로드한 파일 `food10.zip`의 압축을 해제하면 `food10` 디렉토리의 내부에 두 개의 디렉토리 `train`과 `test`가 있고 각각에는 8,000과 2,000장의 음식 이미지가 들어있다. 이미지의 크기는 균일하지 않다. 

모든 이미지 파일의 이름은 0-9 사이의의 정수로 시작된다. 그 정수가 음식의 종류를 표시한다. 다음의 리스트 `classes`는 0-9번 까지의 음식 종류의 명칭이다. 예를 들어 `5_2871066.jpg`는 5로 시작하므로 나초(nachos)이다.

**주의:** 압축해제한 디렉토리 `food10`과 하위 디렉토리 `train` 및 `test`에 `.DS_Store`라는 이름의 숨김 파일이 있을 수 있다. 이 파일들은 모두 찾아내어 삭제해야 한다.

In [ ]:
classes = ['donuts', 'french_fries', 'gyoza', 'hamburger', 'ice_cream', 'nachos', 'pizza', 'ramen', 'risotto', 'sushi']

Colab을 사용할 경우에는 `food10` 디렉토리를 자신의 Google Drive의 적절한 위치에 저장한다. PC에서 실행할할 경우에는 PC의 적절한 위치에 저장한다.

Colab을 사용할 경우 먼저 자신의 Google Drive를 다음과 같이 마운트한다. 이 작업은 로컬 PC에서는 불필요하다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


데이터 디렉토리의 경로를 다음과 같이 설정한다. 각자 자신의 환경에 맞게 설정하면 된다.

In [ ]:
data_root = '/content/drive/MyDrive/DeepLearning2022/datasets/food10'

_________________________________________________________________________

**문제 1:** 이미지의 크기를 균일하게 하고, 픽셀값을 적절하게 정규화하며, 또한 이미지와 라벨을 텐서(Tensor)로 변환하기 위한 데이터 전처리가 필요하다. `torchvision.transforms.Compose`를 이용하여 적절한 전처리 절차 `transform`을 정의하라. 이미지는 50*50 크기로 resizing하고, 각 채널의 픽셀값은 적절하게 정규화한다. 

In [ ]:
transform = transforms.Compose([
    #  YOUR CODE HERE




    ]
    )

_________________________________________________________________________

**문제 2:** 데이터셋을 하나의 클래스 `FoodImageDataset`로 정의하라. 이 클래스를 이용하여 데이터셋을 생성할 때 training 데이터셋은 다음과 같이:

`trainset = FoodImageDataset(data_root, training=True, transform=transform)`

그리고, 테스트 데이터셋은 다음과 같이:

`testset = FoodImageDataset(data_root, training=False, transform=transform)`

생성할 수 있어야 한다. 즉, 매개변수수 `training`이 `True`인지 `False`인지에 따라서 구분하도록 구현하라.

In [ ]:
from PIL import Image

class FoodImageDataset(Dataset):
    # YOUR CODE HERE












    

__________________________________________________________________

**문제 3:** 작성한 클래스 `FoodImageDataset`를 이용하여 두 개의 데이터셋 `trainset`, `testset`과 또한 각각에 대한 데이터 로더 `trainloader`, `testloader`를 생성하라. `trainloader`에서는 데이터를 shuffling하고, `testloader`에서는 shuffling하지 않는다. 배치 크기는 32로 한다.

In [ ]:
batch_size = 32

In [ ]:
# 여기에서 trainset, testset, trainloader, testloader를 생성한다.





_______________________________________________________________________________

아래의 함수 `imshow`는 하나의 Tensor 타입의 이미지를 받아서 화면에 디스플레이하는 함수이다. 이 함수는 픽셀 값들이 `-1.0~1.0` 사이의 실수로 정규화되어 있다고 가정하고 있다. 

In [ ]:
import numpy as np

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

아래의 코드를 실행하면 하나의 이미지가 디스플레이 되어야 한다. 또한 배치(batch)의 shape과 최대값, 최소값, 평균값 등이 올바른지 확인해보라.

In [ ]:
for img, label in trainloader:
    imshow(img[0])
    print(label[0])
    print(img.shape)
    print(label.shape)
    print(img[0].max())
    print(img[0].min())
    print(img[0].mean())
    break

실행할 디바이스를 선택한다.

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


**문제 4:** 3개의 컨볼루션 층을 가진 CNN을 하나의 클래스 Net으로 구현하라. 각각의 컨볼루션 층에서는 순서대로 16, 32, 32개의 크기가 3인 필터를 적용하라. `stride`나 `padding` 등의 나머지 선택사항들은 `nn.Conv2d`의 디폴트 설정을 그대로 따른다. 각각의 컨볼루션 층 다음에는 MaxPooling 층과 ReLu 활성화 함수를 적용하라. 마지막 컨볼루션 층의 출력에 Flatten 층을 적용한 후 적절한 크기의 3개의 Linear 층을 거쳐서 최종 출력이 나오도록 만들어라.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    # YOUR CODE HERE












 직성한 클래스 `Net`을 이용하여 네트워크를 생성하고 디바이스에 할당한다.

In [ ]:
net = Net()
net.to(device)

_______________________________________________________________________________

손실함수와 옵티마이저를 다음과 같이 정의한다.

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

__________________________________________________________________________

**문제 5:** 한 에포크(epoch)의 트레이닝을 수행하는 함수 `train`을 작성하라. 한 에포크 동안의 평균 손실(loss)을 계산하여 출력하도록 구현하라.

In [ ]:
def train(dataloader, model, loss_fn, optimizer)  
    # YOUR CODE HERE








    

____________________________________________________________________________

**문제 6:** 모든 테스트 데이터를 한 번 사용하여 테스트를 수행하는 함수 `test`를 작성하라. 정확도(accuracy)를 계산하여 출력하도록 구현하라. 

In [ ]:
def test(dataloader, model, loss_fn):
    # YOUR CODE HERE











_______________________________________________________________________

아래의 코드는 20 에포크(epoch) 동안 training을 수행한다. 매 epoch마다 평균 손실(loss)이  출력된다. 또한 매 epoch마다 테스트 데이터셋에 속한 모든 이미지를 사용하여 테스트를 수행하고 예측의 정확도를 계산하여 출력한다. 

In [ ]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, net, criterion, optimizer)
    test(testloader, net, criterion)
print("Done!")

___________________________________________________________

**문제 7:** 한 개의 numpy array로 표현된 3채널 이미지와 위에서 학습한 네트워크 모델 net을 입력으로 받아서 음식의 종류를 판정하여 음식 이름과 추정확률을 출력하는 함수 `eval`을 작성하라. 이미지의 사이즈는 50*50이다. `nn.Softmax` 모듈을 이용하라.

In [ ]:
# Softmax 층을 생성한다. 이것을 함수 eval 내부에서 사용하라.
sm = nn.Softmax(dim=1)

# 매개변수 model은 트레이닝된 네트워크 모델이며, img는 50*50*3 크기의 numpy array이며
# 배열에 저장된 값은 0~255 사이의 정수이다.

def eval(model, img):  
    # YOUR CODE HERE
    
    
    
    
    




아래의 코드를 실행하면 예를 들어 다음과 같이 음식 이름과 추정 확률이 출력되어야 한다:

    `ice_cream with probability 0.20791929960250854`


In [ ]:
fakeimg = np.random.randint(256, size=(50, 50, 3))
eval(net, fakeimg)